# Simulations of Survival rate

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd

# Own Classes
from Simulation_Routing import *
from Simulating import *

Initialize the class using the ip of the server.

In [2]:
ip = "3.123.42.170"

In [3]:
Simulations = Simulation(ip)

In [4]:
routing = RoutingSimulation(ip)

In [5]:
print(dir(routing))

['AED_FILE_PATH', 'AED_ISO', 'AED_LAYER', 'CLIENT_ORS', 'IP', '_RoutingSimulation__closest_location', '_RoutingSimulation__closest_location_aed', '_RoutingSimulation__direct_vs_vector', '_RoutingSimulation__directions', '_RoutingSimulation__fastest_comparisson', '_RoutingSimulation__fastest_vector', '_RoutingSimulation__possible_routing_direct', '_RoutingSimulation__possible_routing_indirect', '_RoutingSimulation__send_responders', '_RoutingSimulation__survival_probability', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'fastest_time']


In [6]:
print(dir(Simulations))

['AED_ISO', 'AEDs', 'AMBULANCES', 'CLIENT_ORS', 'IP', 'PATIENTS', 'STAT_SEC_GEOMETRIES', 'VECTORS', '_Simulation__clean_vector', '_Simulation__probability_survival', '_Simulation__survival', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_generate_cfrs', 'average_surv_prob', 'plot_simulation', 'simulation_run']


In [7]:
df = Simulations.simulation_run(0.7, 120, 12, ["Yes"], "day", 0.01)

c:\Users\leonw\anaconda3\Lib\site-packages\geopandas\tools\_random.py:83: FutureWarning: `unary_union` returned None due to all-None GeoSeries. In future, `unary_union` will return 'GEOMETRYCOLLECTION EMPTY' instead.
  return GeoSeries(candidates[:size]).unary_union
c:\Users\leonw\anaconda3\Lib\site-packages\geopandas\tools\_random.py:83: FutureWarning: `unary_union` returned None due to all-None GeoSeries. In future, `unary_union` will return 'GEOMETRYCOLLECTION EMPTY' instead.
  return GeoSeries(candidates[:size]).unary_union
c:\Users\leonw\anaconda3\Lib\site-packages\geopandas\tools\_random.py:83: FutureWarning: `unary_union` returned None due to all-None GeoSeries. In future, `unary_union` will return 'GEOMETRYCOLLECTION EMPTY' instead.
  return GeoSeries(candidates[:size]).unary_union
c:\Users\leonw\anaconda3\Lib\site-packages\geopandas\tools\_random.py:83: FutureWarning: `unary_union` returned None due to all-None GeoSeries. In future, `unary_union` will return 'GEOMETRYCOLLECTIO

Comparing responders vs. vectors


AttributeError: 'Client' object has no attribute 'client'

## Loop Survivability 

The inputs that have to be given are the decline rate of cfrs, the max number of responders send, the hour the cardiac arrest occured, the aeds that are taken into consideration.

Possible aed filters: 
* ["Yes"] = the truly available aeds
* ["Company"] = aeds that are inside a company/manufactury
* ["Private"] = aeds that are private and couldn't be controlled
* Combination of those, ex.: ["Yes","Company"], ["Private","Company"], ["Yes","Company","Private"], etc.
* None = all aeds in the list are considered

In [ ]:
# Number of times the simulations are run to later get an average survivability rate
NO_SIMULATIONS = 10

# Likelihood of a responder declining the call to action
decline_rate = 0.7
# Maximal number of responder contacted
max_number_responder = 120
# Time of the incident
opening_hour = 12.0
# list of values that are in the available column
filter_values = ["Yes"]
# time of the day
if 8.00 <= opening_hour <= 18.00:
    time_of_day = "day"
else:
    time_of_day = "night"
# proportion of CFR of the total (ex.: 0.01 = 1%)
proportion = 0.001

df_final = pd.DataFrame(columns = ['Patient_loc', 'Responder_loc', 'duration_Responder', 'route_Responder',
                                           'Indirect_Responder_loc', 'AED_loc', 'duration_AED', 'route_indirect_Responder',
                                           'Vector_loc', 'duration_Vector', 'route_Vector', 'prob_vec', 'prob_resp',
                                           'filter_values', 'opening_hour', 'decline_rate', 'max_number_responder', 'proportion_of_CFR'])

x = 0
while x < NO_SIMULATIONS:
    df = Simulations.simulation_run(decline_rate, max_number_responder, opening_hour, filter_values, time_of_day, proportion)

    df['filter_values'] = filter_values
    df['opening_hour'] = opening_hour
    df['decline_rate'] = decline_rate
    df['max_number_responder'] = max_number_responder
    df['proportion_of_CFR'] = proportion
    
    df_final = pd.concat([df_final,df])
    df_final = df_final.reset_index(drop = True)
    x += 1

c:\Users\leonw\anaconda3\Lib\site-packages\geopandas\tools\_random.py:83: FutureWarning: `unary_union` returned None due to all-None GeoSeries. In future, `unary_union` will return 'GEOMETRYCOLLECTION EMPTY' instead.
  return GeoSeries(candidates[:size]).unary_union
c:\Users\leonw\anaconda3\Lib\site-packages\geopandas\tools\_random.py:83: FutureWarning: `unary_union` returned None due to all-None GeoSeries. In future, `unary_union` will return 'GEOMETRYCOLLECTION EMPTY' instead.
  return GeoSeries(candidates[:size]).unary_union
c:\Users\leonw\anaconda3\Lib\site-packages\geopandas\tools\_random.py:83: FutureWarning: `unary_union` returned None due to all-None GeoSeries. In future, `unary_union` will return 'GEOMETRYCOLLECTION EMPTY' instead.
  return GeoSeries(candidates[:size]).unary_union
c:\Users\leonw\anaconda3\Lib\site-packages\geopandas\tools\_random.py:83: FutureWarning: `unary_union` returned None due to all-None GeoSeries. In future, `unary_union` will return 'GEOMETRYCOLLECTIO

KeyError: 0

In [2]:
leuven_submunicipalities = ['Leuven (Heverlee)', 'Leuven (Kessel-Lo)', 'Leuven (Wilsele)', 'Leuven (Wijgmaal)', 'Leuven (Korbeek-Lo)', 'Leuven (Haasrode)', 'Leuven (Leuven)', 'LEUVEN', 'Leuven', 'HAASRODE', 'HEVERLEE', 'KESSEL-LO', 'WILSELE', 'WIJGMAAL', 'KORBEEK-LO']

In [4]:
# try for the cad9 dataset
cad_9 = pd.read_parquet("C:/Users/leonw/OneDrive - KU Leuven/Documents/GitHub/AED-Route-Optimization-MDA-Project/Data/cad9.parquet.gzip")

In [5]:
# filter the DataFrame using boolean indexing
filtered_cad_9 = cad_9[cad_9['CityName intervention'].isin(leuven_submunicipalities)]

In [18]:
pd.unique(filtered_cad_9["EventType Trip"])

array(['P039 - CARDIAAL PROBLEEM (NIET PIJN OP DE BORST)',
       'P016 - ZWANGERSCHAP-BEVALLING', 'P068 - UROGENITAAL PROBLEEM',
       'P010 - ADEMHALINGSMOEILIJKHEDEN',
       'P013 - NIET-TRAUMATISCHE RUGPIJN', 'P033 - TRAUMA',
       'P026 - ONWEL/ZIEK ZONDER DUIDELIJKE FOCUS', 'P004 - CVA-TIA',
       'P099 - INTERHOSPITAALTRANSPORT', 'P022 - INTOXICATIE MEDICATIE',
       'P034 - SCHEDELTRAUMA', 'P001 - VERKEERSONGEVAL',
       'P012 - NIET-TRAUMATISCHE BUIKLAST',
       'P031 - PSYCHIATRISCH PROBLEEM', 'P090 - TEST MED',
       'P007 - VAL VAN GROTE HOOGTE (>3 METER)', 'P069 - WONDEN',
       'P098 - COLLOCATIE (NIET GEPLAND)',
       'P019 - BEWUSTELOOS - COMA - SYNCOPE',
       'P015 - EPILEPSIE - STUIPEN', 'P027 - BIJTWONDE',
       'P032 - ALLERGISCHE REACTIE',
       'Y_TI BEVRIJDING PERSOON BEKLEMD/OPGESLOTEN+DGH',
       'P020 - INTOXICATIE ALCOHOL',
       'P002 - AGRESSIE - VECHTPARTIJ - VERKRACHTING',
       'P025 - NIET-TRAUMATISCHE HOOFDPIJN', 'P011 - PIJN OP DE BOR

In [ ]:
P039 - CARDIAAL PROBLEEM (NIET PIJN OP DE BORST),  P008 - PATIËNT MET DEFIBRILLATOR OF PACEMAKER, P003 - HARTSTILSTAND - DOOD - OVERLEDEN

In [7]:
filtered_all = filtered_cad_9[filtered_cad_9["EventType Trip"].isin([
    "P039 - CARDIAAL PROBLEEM (NIET PIJN OP DE BORST)",
    "P008 - PATIËNT MET DEFIBRILLATOR OF PACEMAKER"
])]

In [10]:
filtered_elec = filtered_cad_9[filtered_cad_9["EventType Trip"].isin([
    "P014 - ELEKTROCUTIE", "P028 - VERDRINKING - DUIKONGEVAL"
])]

In [8]:
filtered_cardiac_arrest = filtered_cad_9[filtered_cad_9["EventType Trip"].isin([
    "P003 - HARTSTILSTAND - DOOD - OVERLEDEN"
])]

In [11]:
print(len(filtered_cardiac_arrest))
print(len(filtered_elec))

165
3


In [13]:
import openrouteservice
from openrouteservice import client
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely import geometry
import time
import geopy.distance

import folium
import geopandas as gpd
from shapely.geometry import Point
import random

In [17]:
Check = folium.Map(location=[filtered_all['Latitude intervention'].mean(), filtered_all['Longitude intervention'].mean()], zoom_start=13)
# Add AEDs markers (red)


for _, row in filtered_all.iterrows():
    folium.Marker(
        location=[row['Latitude intervention'], row['Longitude intervention']],
        popup=f"Location",
        icon=folium.Icon(color='green', icon='user', prefix='fa')
    ).add_to(Check)
for _, row in filtered_cardiac_arrest.iterrows():
    folium.Marker(
        location=[row['Latitude intervention'], row['Longitude intervention']],
        popup=f"Location",
        icon=folium.Icon(color='red', icon='user', prefix='fa')
    ).add_to(Check)
for _, row in filtered_elec.iterrows():
     folium.Marker(
         location=[row['Latitude intervention'], row['Longitude intervention']],
         popup=f"Hello",
         icon=folium.Icon(color='orange', icon='heartbeat', prefix='fa')
     ).add_to(Check)

Check